In [ ]:
import os
f = open('openaikey.txt')
os.environ["OPENAI_API_KEY"]=f.read()

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_openai import OpenAI
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA

In [ ]:
# Initialize the LLM 
llm = OpenAI(temperature=0)

# Initialize embeddings using OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
def load_and_embed_document(pdf_path):
    # Load the PDF document
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    # Embed the documents and create a vector store
    vectorstore = FAISS.from_documents(documents, embeddings)
    
    return vectorstore

# Example: Load and embed the standard document
standard_document_path = "resources/DesignDocumentStandards.pdf"
standard_vectorstore = load_and_embed_document(standard_document_path)


In [25]:
def check_compliance(user_document_path, vectorstore):
    # Load the user document using PyPDFLoader
    loader = PyPDFLoader(user_document_path)
    user_doc = loader.load()[0].page_content
    
    # Create a RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=vectorstore.as_retriever())
    
    # Generate compliance checks for each sentence in the user document
    compliance_checks = []
    
    query=f"""
    Given the knowledge base you have help me reivew the content mentioned enclosed between <start> and <end> tag and provide your review comments
    <start>
    {user_doc}
    <end>
    Please also highlight grammatically incorrect statements and abmiguous statements.
    Validate if all the necessary sections are present, specify which sections are missing and how the content should be. Finally review the content and provide the feedback checking the compliance.
    Also give the final verdict as "Approved" or "Rejected"
    """
    
    response = qa_chain.invoke(query)
    return response

# Example: Load and check a user document
user_document_path = "resources/TechnicalDocument.pdf"
compliance_verdicts = check_compliance(user_document_path, standard_vectorstore)
print(compliance_verdicts['result'])



Review Comments:
- The title of the document should be "Technical Design - Abrupt Implementation" instead of "Technical Design - Abrupt implementation".
- The summary section should provide a brief overview of the solution being implemented, rather than just stating that it explains the details of the implementation.
- The document structure section should specify that the document should be organized into the following sections: Title, Reference Table, Summary, Scenarios, Assumptions, Out of Scope, Logical View/Affected Component View, and Process View.
- The reference table section should include a column for the Business Requirement Reference and a column for the UX Design - Figma.
- The summary section should provide a standard requirement of the epic and elaborate on the business driver, rather than just stating that it should talk about the perseverance of the requirement.
- The scenarios section should provide a description of different scenarios where the use case is considere

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=standard_vectorstore.as_retriever())
response=qa_chain.invoke("Generate a sample design document by following the standards with placeholders and sample text specified in the knowledge base ")
print(response['result'])